In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout

In [2]:
# test set 10000 ảnh 0-9 
# data set 60000 ảnh 0-9
(x_train,y_train),(x_test,y_test) = mnist.load_data()

tiền xử lý dữ liệu

In [3]:
# kích thước ảnh 28 x 28
print(x_train.shape) 
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)


(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [4]:
#plot train image
def plot_input_image(i):
  plt.imshow(x_train[i],cmap = 'binary')
  plt.title(y_train[i])
  plt.show()

# for i in range(1):
#   plot_input_image(i)

In [5]:
# tiền xử lý hình ảnh 
# chuẩn hóa về khoảng [0,1]
# giá trị 1 điểm ảnh nằm từ 0 đến 255

x_train = x_train.astype(np.float32)/255
x_test = x_test.astype(np.float32)/255

# đưa dữ liệu hình ảnh  
x_train = np.expand_dims(x_train,-1)
x_test = np.expand_dims(x_test,-1)
print(x_train.shape)



(60000, 28, 28, 1)


In [6]:
# chuyển label về 1 hot vector
y_train = tensorflow.keras.utils.to_categorical(y_train)
y_test = tensorflow.keras.utils.to_categorical(y_test)

In [7]:
model = Sequential()
model.add(Conv2D(32,(3,3),input_shape = (28,28,1),activation='relu'))
model.add(MaxPool2D(2,2))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D(2,2))

model.add(Flatten())
model.add(Dropout(0.25))
#10 class => 10 
model.add(Dense(10,activation='softmax'))


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                1

In [9]:
model.compile(optimizer='adam',
              loss = keras.losses.categorical_crossentropy,
              metrics='accuracy')

In [10]:
# callback
from keras.callbacks import EarlyStopping, ModelCheckpoint

# earlystopping
# monitor : thang đánh giá 
# min_delta: độ tăng tối thiểu để đc coi là chất lượng tăng
# patience: sau 4 lần epoch mà chất lượng models ko tăng thì dừng

es = EarlyStopping(monitor = 'val_accuracy',min_delta = 0.01, patience = 4,verbose = 1)

# model checkpoint
mc = ModelCheckpoint('./mymodel.h5',monitor='val_accuracy',verbose = 1,save_best_only = True)

cb = [es,mc]
# cb =[mc]


huấn luyện mô hình

In [11]:
hi = model.fit(x_train,y_train,epochs = 20,validation_split=0.3,callbacks = cb)

Epoch 1/20
1313/1313 [==============================] - 28s 20ms/step - loss: 0.4762 - accuracy: 0.8545 - val_loss: 0.0819 - val_accuracy: 0.9767

Epoch 00001: val_accuracy improved from -inf to 0.97667, saving model to .\mymodel.h5
Epoch 2/20
1313/1313 [==============================] - 29s 22ms/step - loss: 0.0768 - accuracy: 0.9774 - val_loss: 0.0553 - val_accuracy: 0.9833

Epoch 00002: val_accuracy improved from 0.97667 to 0.98333, saving model to .\mymodel.h5
Epoch 3/20
1313/1313 [==============================] - 26s 20ms/step - loss: 0.0561 - accuracy: 0.9816 - val_loss: 0.0508 - val_accuracy: 0.9847

Epoch 00003: val_accuracy improved from 0.98333 to 0.98472, saving model to .\mymodel.h5
Epoch 4/20
1313/1313 [==============================] - 29s 22ms/step - loss: 0.0458 - accuracy: 0.9863 - val_loss: 0.0480 - val_accuracy: 0.9867

Epoch 00004: val_accuracy improved from 0.98472 to 0.98667, saving model to .\mymodel.h5
Epoch 5/20
1313/1313 [==============================] - 27s

In [12]:
model_S = keras.models.load_model('D:\project1\mymodel.h5')

In [13]:
score = model_S.evaluate(x_test,y_test)
print(score)

313/313 [==============================] - 2s 5ms/step - loss: 0.0364 - accuracy: 0.9883
[0.036394719034433365, 0.9883000254631042]
